# Sampling

In [2]:
from functools import partial
import os

import jax
import jax.numpy as jnp

import matplotlib.pyplot as plt

from optax import EmaState, EmptyState, ScaleByAdamState, ScaleByScheduleState

from diffuse.conditional import CondSDE
from diffuse.filter import generate_cond_sample
from diffuse.sde import SDE, SDEState, LinearSchedule
from diffuse.unet import UNet
from vraie_vie.create_dataset import WMH
from vraie_vie.utils import generate_mask, slice_fourier, slice_inverse_fourier

In [ ]:
config = {
        "modality": "FLAIR",
        "slice_size_template": 49,
        "begin_slice": 26,
        "flair_template_path": "/lustre/fswork/projects/rech/hlp/uha64uw/aistat24/WMH/MNI-FLAIR-2.0mm.nii.gz",
        "path_dataset": "/Users/geoffroyoudoumanessah/Documents/these/projects/datasets/WMH", # "/lustre/fswork/projects/rech/hlp/uha64uw/aistat24/WMH",
        "save_path": "/lustre/fswork/projects/rech/hlp/uha64uw/aistat24/WMH/models/",
        "n_epochs": 4000,
        "batch_size": 32,
        "num_workers": 0,
        "n_t": 32,
        "tf": 2.0,
        "lr": 2e-4,
    }

In [ ]:
# Retrieve trained Parameters

checkpoint = jnp.load(os.path.join(config["save_path"], "ann_215.npz"), allow_pickle=True)

params = checkpoint["params"].item()

ema_state = EmaState(count=checkpoint["ema_state"][0], ema=checkpoint["ema_state"][1])

opt_state = (
    EmptyState(),
    (
        ScaleByAdamState(
            count=checkpoint["opt_state_2"][0], mu=checkpoint["opt_state_2"][1], nu=checkpoint["opt_state_2"][2]
        ),
        ScaleByScheduleState(checkpoint["opt_state_3"][0]),
    ),
)

# Get the Datasets
wmh = WMH(config)
wmh.setup()
train_loader = wmh.get_train_dataloader().dataset

# Get the ScoreNet
nn_unet = UNet(config["tf"] / config["n_t"], 64, upsampling="pixel_shuffle")

def nn_score_(x, t, scoreNet, params):
    return scoreNet.apply(params, x, t)

nn_score = partial(nn_score_, scoreNet=nn_unet, params=params)

In [ ]:
n_steps = 1000
key = jax.random.PRNGKey(0)

ts = jnp.array([config["tf"]])
dts = jnp.array([2.0 / n_steps] * (n_steps))

beta = LinearSchedule(b_min=0.02, b_max=5.0, t0=0.0, T=2.0)
sde = SDE(beta=beta)

for _ in range(10):
    key, subkey = jax.random.split(key)
    init_samples = jax.random.normal(subkey, train_loader[0].shape)
    state_f = SDEState(position=init_samples, t=ts)
    
    revert_sde = partial(sde.reverso, score=nn_score, dts=dts)
    
    key, subkey = jax.random.split(key)
    state_0, state_Ts = revert_sde(subkey, state_f)
    
    plt.imshow(state_Ts.position[-1][..., 0], cmap="gray")
    plt.show()

In [ ]:
# On bruite
key = jax.random.PRNGKey(0)

x0_samples = jnp.array([train_loader[k] for k in range(config['batch_size'])])
n_x0 = x0_samples.shape[0]

key, subkey = jax.random.split(key)
ts = jax.random.uniform(key, (config['n_t'] - 1, 1), minval=1e-5, maxval=config['tf'])
ts = jnp.concatenate([ts, jnp.array([[config['tf']]])], axis=0)

state_0 = SDEState(x0_samples, jnp.zeros((n_x0, 1)))
keys_x = jax.random.split(key, n_x0)
state = jax.vmap(sde.path, in_axes=(0, 0, 0))(keys_x, state_0, ts)

# On sample
n_steps = 1000

ts = jnp.array([config["tf"]])
dts = jnp.array([2.0 / n_steps] * (n_steps))

beta = LinearSchedule(b_min=0.02, b_max=5.0, t0=0.0, T=2.0)
sde = SDE(beta=beta)

key, subkey = jax.random.split(key)
init_samples = state.position[30]
state_f = SDEState(position=init_samples, t=ts)

revert_sde = partial(sde.reverso, score=nn_score, dts=dts)

key, subkey = jax.random.split(key)
state_0, state_Ts = revert_sde(subkey, state_f)

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(x0_samples[30][..., 0], cmap="gray")

plt.subplot(1, 2, 2)
plt.imshow(state_Ts.position[-1][..., 0], cmap="gray")
plt.show()

# Inverse problem

In [ ]:
# Get the Datasets
wmh = WMH(config)
wmh.setup()
train_loader = wmh.get_train_dataloader().dataset

key = jax.random.PRNGKey(0)

x = train_loader[0][..., 0].numpy()

key, subkey = jax.random.split(key)
w = jax.random.randint(subkey, shape=x.shape, minval=1, maxval=301)
s = 2000

mask = generate_mask(key, w, s)

fourier_transform_x = slice_fourier(x)
y = mask * fourier_transform_x

reconstructed_x = slice_inverse_fourier(y)

fig, axs = plt.subplots(1, 4, figsize=(20, 5))
axs[0].imshow(x, cmap='gray')
axs[0].set_title('Original x')
axs[1].imshow(jnp.abs(fourier_transform_x), cmap='gray', vmin=0, vmax=jnp.percentile(jnp.abs(fourier_transform_x), 99))
axs[1].set_title('Fourier Transform of x')
axs[2].imshow(jnp.abs(y), cmap='gray', vmin=0, vmax=jnp.percentile(jnp.abs(y), 99))
axs[2].set_title('y (Masked Fourier Transform)')
axs[3].imshow(jnp.abs(slice_inverse_fourier(y)), cmap='gray')
axs[3].set_title('Inverse Fourier of y')
plt.tight_layout()
plt.show()




# Conditional

In [ ]:
# Load dataset
wmh = WMH(config)
wmh.setup()
train_loader = wmh.get_train_dataloader().dataset

key = jax.random.PRNGKey(0)
beta = LinearSchedule(b_min=0.02, b_max=5.0, t0=0.0, T=2.0)

checkpoint = jnp.load(os.path.join(config["save_path"], "ann_1720.npz"), allow_pickle=True)


nn_unet = UNet(config["tf"] / config["n_t"], 64, upsampling="pixel_shuffle")
params = checkpoint["params"].item()

def nn_score_(x, t, scoreNet, params):
    return scoreNet.apply(params, x, t)

nn_score = partial(nn_score_, scoreNet=nn_unet, params=params)

sde = SDE(beta=beta)

key, subkey = jax.random.split(key)
masking = lambda w: generate_mask(subkey, w, s)
s = 2000

def measure(x, w, masking):
    mask = masking(w)
    return mask * slice_fourier(x[..., 0])

def restore(w, x, masking, measured):
    inv_mask = 1 - masking(w)
    return inv_mask * slice_inverse_fourier(x[..., 0]) + measured

x = jnp.array(train_loader[0].numpy())
w = jax.random.randint(subkey, shape=x[..., 0].shape, minval=1, maxval=301)
y = measure(x, w, masking)

cond_sde = CondSDE(beta=beta, mask=masking, measure=measure, restore=restore, tf=2.0, score=nn_score)

key, subkey = jax.random.split(key)
res = generate_cond_sample(y, w, subkey, cond_sde, x.shape, 300, 200)